## AI-powered Contact Center Intake (AICCI) Agents - Memory Design

 * Author:    Kyle Zarifsadr
 * Created:   January 2025

In [1]:
from dotenv import load_dotenv
load_dotenv() 
import numpy as np
import pandas as pd
import json
import os
import openai
import random
import time
from datetime import datetime
from openai import AzureOpenAI
import psycopg
from sentence_transformers import SentenceTransformer
import autogen
from autogen import GroupChat
from autogen import GroupChatManager
from autogen import AssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
from autogen import ConversableAgent, UserProxyAgent, config_list_from_json
from autogen.retrieve_utils import TEXT_FORMATS
import psycopg2
from psycopg2.extras import execute_values
from pgvector.psycopg2 import register_vector
from autogen import AssistantAgent, ConversableAgent, UserProxyAgent

/usr/local/Cellar/jupyterlab/4.2.1/libexec/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.


### Azure OpenAI

Reference: https://microsoft.github.io/autogen/0.2/docs/reference/agentchat/contrib/vectordb/pgvectordb

In [2]:
llm_config = {
    "config_list": [
        {
            "model": "gpt-4o",
            "api_key": os.getenv("OPENAI_API_KEY"),
            "api_type": "azure",
            "base_url": os.getenv("OPENAI_API_BASE"),
            "api_version": os.getenv("API_VERSION"),
        },
    ],
    "temperature": 0.0,
    "timeout": 300,
}

In [3]:
load_dotenv()  

api_key = os.getenv("OPENAI_API_KEY")
azure_endpoint = os.getenv("OPENAI_API_BASE")
api_version = os.getenv("API_VERSION")
model = "gpt-4o"
temprature = 0.0
top_p = 1.0

def get_response_client(template, text, temprature=temprature, top_p=top_p, model=model, azure_endpoint=azure_endpoint, api_key=api_key, api_version=api_version):
    time.sleep(1)

    client = AzureOpenAI(
        azure_endpoint = azure_endpoint, 
        api_key= api_key,  
        api_version= api_version
    )

    conversation = [
        {"role": "system", "content": template},
        {"role": "user", "content": text},
    ]

    response = client.chat.completions.create(
        model=model,
        temperature=temprature,
        top_p= top_p,
        max_tokens=4000,
        response_format={ "type": "json_object" },  #JSON mode
        messages=conversation
    )

    return response.choices[0].message.content

### Simulated API Calls to PostgreSQL

In [4]:
def get_db_connection():
    conn = psycopg2.connect(
        dbname="test",
        user="postgres",
        password="test_password",
        host="localhost",
        port="5433"
    )
    return conn

def make_request(url, clientKey):
    conn = get_db_connection()
    cur = conn.cursor()

    query = "SELECT response_data FROM guidelines WHERE url = %s AND clientKey = %s;"
    cur.execute(query, (url, clientKey))
    result = cur.fetchone()
    
    if result:
        response_data = result[0]
        print(f"Response from {url}: {json.dumps(response_data, indent=2)}")
        return response_data
    #     store_request_response(url, params, response_data)
    # else:
    #     print(f"No data found for URL: {url}")
    #     store_request_response(url, params, {"error": "Not Found"})
    
    cur.close()
    conn.close()

### Test clientKey

In [5]:
clientKey = 'Call Center Training 32 - Target Corporation'  # [ 'Call Center Training 32 - Target Corporation' ,'Call Center Training 31 - UPS', 'StarSpaces', 'BankCorp', 'HealthCarePlus', 'EduCare',]

In [6]:
url = "/v6.0/Intake/guidelines"
guidelines = make_request(url, clientKey)

Response from /v6.0/Intake/guidelines: "{\"clientKey\": \"Call Center Training 32 - Target Corporation\", \"CustomText\": \"\\nOpening\\n\\nThank you for calling the Call Center Training 32 - Target Corporation Hotline, this is (CS Name). Would you like to file a report or follow up on an existing report? Finished\\n\\nDON'T READ: Factory workers may report on this line. Accept reports from factory workers who work for or with Call Center Training 32 - Target Corporation.\\nREAD: Before we begin, please know that this process may take 10-15 minutes or more depending on the nature of your concern or question. To ensure proper submission of your report, you will need to remain on the line until I\\u2019ve read the report narrative details back to you. I will also provide you with a report key for follow-up purposes. Do you wish to proceed?\\n\\nDON'T READ: Please accept reports from Starbucks employees that work inside Target retail stores.\\nDON'T READ: Please accept reports from SHIPT 

In [7]:
url = "/v6.0/Intake/locations"
locations = make_request(url, clientKey)

Response from /v6.0/Intake/locations: "{\"clientKey\": \"Call Center Training 32 - Target Corporation\", \"Locations\": [{\"Case_CompanyLocation\": \"Branch Office D\", \"Case_CompanyCity\": \"Austin\", \"Case_CompanyState\": \"TX\", \"Case_CompanyZip\": \"73301\", \"Case_CompanyCountry\": \"USA\", \"Case_LocationCustomField1\": \"Technical Support\", \"Case_LocationCustomField2\": \"SW\", \"Case_LocationCustomField3\": \"District 7\", \"Case_LocationCustomField4\": \"Floor 5\"}, {\"Case_CompanyLocation\": \"Branch Office E\", \"Case_CompanyCity\": \"Denver\", \"Case_CompanyState\": \"CO\", \"Case_CompanyZip\": \"80201\", \"Case_CompanyCountry\": \"USA\", \"Case_LocationCustomField1\": \"Sales\", \"Case_LocationCustomField2\": \"MW\", \"Case_LocationCustomField3\": \"District 4\", \"Case_LocationCustomField4\": \"Building 1\"}, {\"Case_CompanyLocation\": \"Branch Office F\", \"Case_CompanyCity\": \"Orlando\", \"Case_CompanyState\": \"FL\", \"Case_CompanyZip\": \"32801\", \"Case_Company

In [8]:
url = "/v6.0/Intake/GetMobileIssueTypesWithDefaults"
IssueTypes = make_request(url, clientKey)

Response from /v6.0/Intake/GetMobileIssueTypesWithDefaults: "{\"clientKey\": \"Call Center Training 32 - Target Corporation\", \"data\": {\"ViolationTypeList\": [{\"Description\": \"Unauthorized access to company systems or data, hacking, or other cyber threats. This includes phishing attempts, malware infections, and other cybersecurity incidents.\", \"Name\": \"Cybersecurity Breach\", \"ViolationTypeId\": 13}, {\"Description\": \"Violations related to environmental regulations, including improper disposal of hazardous materials, pollution, and non-compliance with environmental laws.\", \"Name\": \"Environmental Violation\", \"ViolationTypeId\": 14}, {\"Description\": \"Reports of physical violence or threats of violence in the workplace. This includes fights, assaults, and other forms of physical aggression.\", \"Name\": \"Workplace Violence\", \"ViolationTypeId\": 15}, {\"Description\": \"Instances of theft, embezzlement, or other forms of financial fraud. This includes stealing com

In [9]:
url = "/v6.0/ViolationQuestion/GetViolationQuestionAndAnswersForPlatformPackages"
questions = make_request(url, clientKey)

Response from /v6.0/ViolationQuestion/GetViolationQuestionAndAnswersForPlatformPackages: "{\"clientKey\": \"Call Center Training 32 - Target Corporation\", \"data\": [{\"violationTypeId\": 13, \"ViolationQuestionId\": 171, \"Question\": \"Where did this happen?\"}, {\"violationTypeId\": 13, \"ViolationQuestionId\": 172, \"Question\": \"What type of cyber threat occurred?\"}, {\"violationTypeId\": 13, \"ViolationQuestionId\": 173, \"Question\": \"Who was involved in the cybersecurity breach?\"}, {\"violationTypeId\": 14, \"ViolationQuestionId\": 171, \"Question\": \"Where did this happen?\"}, {\"violationTypeId\": 14, \"ViolationQuestionId\": 174, \"Question\": \"What environmental regulation was violated?\"}, {\"violationTypeId\": 14, \"ViolationQuestionId\": 175, \"Question\": \"Who was responsible for the environmental violation?\"}, {\"violationTypeId\": 15, \"ViolationQuestionId\": 171, \"Question\": \"Where did this happen?\"}, {\"violationTypeId\": 15, \"ViolationQuestionId\": 17

### 1. Process Guidelines Data
Tagging and Parsing Approach


In [10]:
template_process_guidelines = """
Extract text in JSON from text or HTML elements similar to the following:

{
  "OPENING": "Text from opening HTML element",
  "READ": {
    "READ_1": "1st Text to read",
    "READ_2": "2nd Text to read",
  },
  "DONT_READ": {
    "DONT_READ_1": "1st Text not to read",
    "DONT_READ_2": "2nd Text not to read",
  },
  "Other_Instructions": {
    "new_report": "Other instructions for new report",
    "follow_up": "Other instructions for follow-up"
  }
}

Do not wrap the output in quotation marks.
Do not wrap the output in code block delimiters (```).
"""

In [11]:
guidelines = get_response_client(template_process_guidelines, guidelines)
guidelines = json.loads(guidelines)
print(guidelines)

{'OPENING': 'Thank you for calling the Call Center Training 32 - Target Corporation Hotline, this is (CS Name). Would you like to file a report or follow up on an existing report?', 'READ': {'READ_1': 'Before we begin, please know that this process may take 10-15 minutes or more depending on the nature of your concern or question. To ensure proper submission of your report, you will need to remain on the line until I’ve read the report narrative details back to you. I will also provide you with a report key for follow-up purposes. Do you wish to proceed?'}, 'DONT_READ': {'DONT_READ_1': 'Factory workers may report on this line. Accept reports from factory workers who work for or with Call Center Training 32 - Target Corporation.', 'DONT_READ_2': 'Please accept reports from Starbucks employees that work inside Target retail stores.', 'DONT_READ_3': 'Please accept reports from SHIPT employees.', 'DONT_READ_4': 'Do not accept reports for CVS employees.'}}


In [12]:
opening_count = 2 if "OPENING" in guidelines else 1

if isinstance(guidelines.get("READ"), dict):
    read_count = len(guidelines["READ"])  
else:
    read_count = 1  

total_count = opening_count + read_count
print(f"Total count for group chat: {total_count}")

Total count for group chat: 3


### 2. Imminent Issues

In [13]:
template_imminent_issue = """
At any point during the interview, AI detects imminent issue and flags report with priority with this message: "Flag report with priority"
At any point during the interview, (If suicide ideations deteted, transfer call to CS workflow) with this message: "transfer call to CS workflow"

Examples:
Threats of Violence: Reports of threats of physical harm or violence from coworkers, supervisors, or customers.
Harassment or Assault: Incidents of sexual harassment, physical assault, or severe bullying.
Medical Emergencies: Situations involving severe health issues such as heart attacks, severe allergic reactions, or injuries.
Unsafe Working Conditions: Immediate dangers due to unsafe conditions like hazardous materials, lack of safety equipment, or structural hazards.
Fire or Explosion: Reports of fires, explosions, or similar emergencies posing immediate threats to safety.
Active Shooter or Hostage Situation: Incidents involving an active shooter or hostage situation within the workplace.
Severe Psychological Distress: Cases of severe psychological distress or suicidal thoughts/intentions.

(If suicide ideations deteted, transfer call to CS workflow) with this message: "transfer call to CS workflow"

Do not wrap the output in quotation marks.
Do not wrap the output in code block delimiters (```).
"""

### 3. Guidelines

#### 3.a. Guidelines: Agents

In [14]:
template_guidelines = """
Role: Ethics and Compliance Contact Center Agent
Objective: You will be given the callers [previous_chat_history] and applicable [guidelines]. 
Your primary responsibility is to determine the call reasons "New Report"/ "Follow-Up" after gathering all answers to the [guidelines] READ messages/questions in a valid JSON.

Don't repeat the content of the previous_chat_history.
Under no circumstances determine the call reasons if you don't have an answer to the READ messages/questions.
Your response should consist of only one question at a time retrieved from the READ messages/questions.
You will categorize the call as either a new report or a follow-up on an existing case.

Call Reasons:
New Report: The caller is calling to file a new report.
Follow-Up: The caller is calling to follow up on an existing case.

Steps to Determine Call Reason:
Always start with the "Opening" text from the [guidelines]. check if the "Opening" statement has already been used in the current session 
and then proceed to the next relevant READ messages/questions.
Finally determine the call reason: "New Report"/ "Follow-Up"

{"call_reason":"Follow-Up"}

Do not wrap the output in quotation marks.
Do not wrap the output in code block delimiters (```).
"""

In [15]:
user_agent = ConversableAgent(
    name="user_agent",
    system_message= f"You are an assistant + {template_imminent_issue}",
    llm_config=llm_config,
    human_input_mode= "NEVER",
)

guidelines_agent = ConversableAgent(
    name="guidelines_agent",
    system_message= f"Report Datetime: {template_guidelines} + guidelines: {guidelines} + {template_imminent_issue}",
    llm_config=llm_config,
    human_input_mode= "NEVER",
)

#### 3.b. Guidelines: Group Chat

In [16]:
def state_transition_guidelines(last_speaker, groupchat):
    messages = groupchat.messages

    if last_speaker is user_agent:
        return guidelines_agent
    elif last_speaker is guidelines_agent:
        return None

groupchat_guidelines = autogen.GroupChat(
    agents=[user_agent,
            guidelines_agent
           ],
    messages=[],
    max_round=20,
    speaker_selection_method=state_transition_guidelines,
)

manager_guidelines = autogen.GroupChatManager(groupchat=groupchat_guidelines, llm_config=llm_config)

#### 3.c. Guidelines: Functions

In [17]:
# prompt the user to input data in JSON format and collect that data into a dictionary
def collect_json_data():
    follow_ups = {}
    while not follow_ups:
        user_input = input("Please provide follow-up data in JSON format (e.g. {\"key\": \"value\"} or {\"key1\": \"value1\", \"key2\": \"value2\"}): ")
        try:
            user_data = json.loads(user_input)

            if isinstance(user_data, dict):
                follow_ups.update(user_data)
            else:
                print("The JSON input must be an object (key-value pairs). Please try again.")
        except json.JSONDecodeError:
            print("Invalid JSON input. Please try again.")
    
    return follow_ups 

# parse the content of the last message in the memory list as JSON and returns the parsed JSON object where each question is a key in a dictionary with an empty string as its value.
def extract_questions_from_memory(memory):
    last_message = memory[-1]
    content = last_message["content"]

    try:
        content_dict = json.loads(content)
        return content_dict
    except json.JSONDecodeError:
        questions = [q.strip() + "?" if not q.strip().endswith("?") else q.strip() for q in content.split("\n") if q.strip()]
        questions_dict = {q: "" for q in questions}
        return json.dumps(questions_dict, indent=4)

def read_chat_history():
    try:
        with open("memory.json", "r") as json_file:
            try:
                return json.load(json_file)
            except json.JSONDecodeError:
                return []  # Initialize as an empty list if invalid or empty JSON
    except FileNotFoundError:
        return []  # If file doesn't exist, initialize as empty list

def write_chat_history(chat_history_data):
    with open("memory.json", "w") as json_file:
        json.dump(chat_history_data, json_file, indent=4)

def extract_and_print_questions():
    with open("memory.json", "r") as test_file:
        memory = json.load(test_file)
    questions = extract_questions_from_memory(memory)
    print(questions)

In [18]:
def main_guidelines():
    last_message = {}

    user_agent.initiate_chat(recipient=manager_guidelines, message=str(last_message), clear_history=False)
    messages_json = manager_guidelines.messages_to_string(manager_guidelines.groupchat.messages)

    chat_history_data = read_chat_history()

    new_messages = json.loads(messages_json)  
    chat_history_data.extend(new_messages) 

    write_chat_history(chat_history_data)
    extract_and_print_questions()

def main_guidelines_follow_up():
    with open("memory.json", "r") as test_file:
        memory = json.load(test_file)

    new_message = collect_json_data()
    memory.append(new_message)
    last_message = json.dumps(memory)

    user_agent.initiate_chat(recipient=manager_guidelines, message=last_message, clear_history=False)
    messages_json = manager_guidelines.messages_to_string(manager_guidelines.groupchat.messages)

    chat_history_data = read_chat_history()

    new_messages = json.loads(messages_json)  
    chat_history_data.extend(new_messages)  

    write_chat_history(chat_history_data)
    extract_and_print_questions()

#### 3.d. Guidelines: Call Simulation

In [19]:
# if __name__ == "__main__":
#     main_guidelines()
#     main_guidelines_follow_up()
#     main_guidelines_follow_up()

### 4. Locations

#### 4.a. Locations: Agents

In [20]:
template_locations = """
Retrieve the location of the event from [locations] by asking a few questions one by one about the state, city, etc., 
using [locations] data.

Do not repeat questions with the same context.

As soon as you detect the location from [locations], respond with a question to confirm "Case_CompanyLocation": "", 
and "Case_CompanyCity": "" from the data package available in [locations]. 
The final response after confirmation will be:

Example confirmation response: "Did the event occur in "Branch Office C"?

Example final response:

{
      "Case_CompanyLocation": "Branch Office C",
      "Case_CompanyCity": "Albany",
      "Case_CompanyState": "NY",
      "Case_CompanyZip": "12207",
      "Case_CompanyCountry": "USA",
      "Case_LocationCustomField1": "Customer Service",
      "Case_LocationCustomField2": "SE",
      "Case_LocationCustomField3": "District 3",
      "Case_LocationCustomField4": "Suite 200"
    }

Do not wrap the output in quotation marks.
Do not wrap the output in code block delimiters (```).
"""

In [21]:
locations_agent = ConversableAgent(
    name="locations",
    system_message= f"locations: {locations} + {template_locations} + {template_imminent_issue}",
    llm_config=llm_config,
    human_input_mode= "NEVER",
)

#### 4.b. Locations: Group Chat

In [22]:
def state_transition_locations(last_speaker, groupchat):
    messages = groupchat.messages

    if last_speaker is user_agent:
        return locations_agent
    elif last_speaker is locations_agent:
        return None

groupchat_locations = autogen.GroupChat(
    agents=[user_agent,
            locations_agent
           ],
    messages=[],
    max_round=10,
    speaker_selection_method=state_transition_locations,
)

manager_locations = autogen.GroupChatManager(groupchat=groupchat_locations, llm_config=llm_config)

#### 4.c. Locations: Functions

In [23]:
def main_locations():
    last_message = {}

    user_agent.initiate_chat(recipient=manager_locations, message=str(last_message), clear_history=False)
    messages_json = manager_locations.messages_to_string(manager_locations.groupchat.messages)

    chat_history_data = read_chat_history()

    new_messages = json.loads(messages_json)  
    chat_history_data.extend(new_messages) 

    write_chat_history(chat_history_data)
    extract_and_print_questions()

def main_locations_follow_up():
    with open("memory.json", "r") as test_file:
        memory = json.load(test_file)

    new_message = collect_json_data()
    memory.append(new_message)
    last_message = json.dumps(memory)

    user_agent.initiate_chat(recipient=manager_locations, message=last_message, clear_history=False)
    messages_json = manager_locations.messages_to_string(manager_locations.groupchat.messages)

    chat_history_data = read_chat_history()

    new_messages = json.loads(messages_json)  
    chat_history_data.extend(new_messages)  

    write_chat_history(chat_history_data)
    extract_and_print_questions()

#### 4.d. Locations: Call Simulation

In [24]:
# if __name__ == "__main__":
#     main_locations()
#     main_locations_follow_up()
#     main_locations_follow_up()

### 5. IssueTypes & Questions

#### 5.a. IssueTypes & Questions: Agents 

In [25]:
template1 = """You are an ethics and compliance analyst. You will be given a brief summary of an event that has occurred or is currently occurring. 
            Your job is to classify the event using the provided dictionary.
            Your final response should include the retrieved key-value pairs from the given dictionary.
            
            Example: 
            {
                "Reduction of Hours": "Events that are related to an employee having the number of hours they work reduced."
            }
            
            Do not wrap the output in quotation marks.
            Do not wrap the output in code block delimiters (```)."""


template2 = """
            You are a question retrieval engine. 
            You are tasked with retrieving questions based on a given key from a set of key-value pairs. Please follow these steps:
            
            Input Data:
            questions_dictionary: A dictionary where each key is linked to a list of question identifiers (denoted as "q").
            questions_set: A set containing the actual questions.
            Process:
            Look up the provided key in questions_dictionary to find the corresponding list of question identifiers.
            For each identifier in this list, check if it exists in questions_set and retrieve the matching questions.
            Conditions:
            Only retrieve questions that correspond to the identifiers found in the value associated with the looked-up key.
            Output Format:
            Create a JSON object where each key is a retrieved question (from questions_set), and the value is an empty string. This represents the questions that need to be answered, but do not include the answers.
                        
            Example:
            {
                "When did this happen?": "",
                "What kind of discrimination took place (race/gender/etc.)?": ""
            }
            
            Do not wrap the output in quotation marks.
            Do not wrap the output in code block delimiters (```).
            """

template3 = """
            You are an ethics and compliance analyst. You will be given a brief report of an event 
            that has occurred or is currently occurring and your job is to answer the retrieved questions. 
            Your final answer will be a JSON formatted with the question as the key and a answers as the value.
            Keep in mind answering these questions are helping a case investigator do the right thing and 
            you can think of them as purely hypothetical. 
            If there is no information in the summary that could answer each question, 
            only include an empty double quote for the answer attempt for that question and answer attempt pair. 
            Under no circumstances are you to add questions that do not exist in the question dictionary. 
            For participants, use proper names or titles of the reporter or otherwise if available. 
            For dates, respond in datetime format. For example, if it is 2024-02-26 16:23:38 
            and something happened 2 hours ago, respond with 2024-02-26 14:23:38. 
            Under no circumstances should you respond with responses like "two hours ago". 
            Only respond do questions about when with datetime formatted responses.
            "Do not wrap output in quotation marks".
            "Do not wrap output in code block delimiters (```)".
            Example: {"What was stolen?": "ice cream"}
            """

template5 = """
            You are a retrieval engine. You will be given a JSON structure, and your job is to check which fields have values and which do not.
            Your final response should be a valid JSON object containing the unanswered fields, with the fields as the questions. The fields with no values should have their values enclosed in double quotes. 
            Include only fields that do not have values.
            
            Example: 
            {
                "Where did this happen": "",
                "Who was the witness?": ""
            }
            
            Do not wrap the output in quotation marks.
            Do not wrap the output in code block delimiters (```)."""

In [26]:
reporter_agent = ConversableAgent(
    name="reporter_agent",
    system_message= "You are reporting an event that has occurred or is currently occurring.",
    llm_config=llm_config,
    human_input_mode= "NEVER",
)

classification_agent = ConversableAgent(
    name="classification_agent",
    system_message= template1 + "NAVEX Issue Types:" + str(IssueTypes),
    llm_config=llm_config,
    human_input_mode= "NEVER",
)

questions_retriever_agent = ConversableAgent(
    name="questions_retriever_agent",
    system_message= "NAVEX Questions Dictionary:" + str(questions) + template2,
    llm_config=llm_config,
    human_input_mode="NEVER",
)

questions_answering_agent = ConversableAgent(
    name="questions_answering_agent",
    system_message= template3,
    llm_config=llm_config,
    human_input_mode="NEVER",
)

unanswered_questions_agent = ConversableAgent(
    name="unanswered_questions_agent",
    system_message= template5,
    llm_config=llm_config,
    human_input_mode= "NEVER",
)

#### 5.a. IssueTypes & Questions: Group Chat

In [27]:
def state_transition_issue_questions(last_speaker, groupchat):
    messages = groupchat.messages

    if last_speaker is reporter_agent:
        return classification_agent
    elif last_speaker is classification_agent:
        return questions_retriever_agent
    elif last_speaker is questions_retriever_agent:
        return questions_answering_agent
    elif last_speaker is questions_answering_agent:
        return unanswered_questions_agent
    elif last_speaker is unanswered_questions_agent:
        return None

groupchat_issue_questions = autogen.GroupChat(
    agents=[reporter_agent,
            classification_agent, 
            questions_retriever_agent, 
            questions_answering_agent,
            unanswered_questions_agent
           ],
    messages=[],
    max_round=20,
    speaker_selection_method=state_transition_issue_questions,
)

manager_issue_questions = autogen.GroupChatManager(groupchat=groupchat_issue_questions, llm_config=llm_config)

#### 5.c. IssueTypes & Questions: Functions

In [28]:
def convert_json_to_code(json_input):
    try:
        data = json.loads(json_input)
        code_lines = []

        for key, value in data.items():
            value_str = value if isinstance(value, str) else json.dumps(value)
            code_lines.append(f'{key} = {value_str}')

        return "\n".join(code_lines)
    
    except json.JSONDecodeError:
        return "Invalid JSON provided to the code conversion function."
        

def collect_transcription_data():
    transcription = {"transcription": ""}

    while not transcription["transcription"]:
        user_input = input("Please provide transcription text in JSON format (e.g. {\"transcription\": \"I want to start filing a report!\"}): ")
        try:
            user_data = json.loads(user_input)
            transcription.update(user_data)
        except json.JSONDecodeError:
            print("Invalid JSON input. Please try again.")
    
    transcription_text = convert_json_to_code(json.dumps(transcription))
    
    return transcription_text

In [29]:
def main_issue_questions():
    last_message = collect_transcription_data() 

    reporter_agent.initiate_chat(recipient=manager_issue_questions, message=str(last_message), clear_history=False)
    messages_json = manager_issue_questions.messages_to_string(manager_issue_questions.groupchat.messages)
    
    chat_history_data = read_chat_history()

    new_messages = json.loads(messages_json)  
    chat_history_data.extend(new_messages) 

    write_chat_history(chat_history_data)
    extract_and_print_questions()


def main_issue_questions_follow_up():
    with open("memory.json", "r") as test_file:
        memory = json.load(test_file)

    new_message = collect_json_data()
    memory.append(new_message)
    last_message = json.dumps(memory)

    user_agent.initiate_chat(recipient=manager_guidelines, message=last_message, clear_history=False)
    messages_json = manager_guidelines.messages_to_string(manager_guidelines.groupchat.messages)

    chat_history_data = read_chat_history()

    new_messages = json.loads(messages_json) 
    chat_history_data.extend(new_messages) 

    write_chat_history(chat_history_data)
    extract_and_print_questions()

#### 5.d. IssueTypes & Questions: Call Simulation

In [30]:
# if __name__ == "__main__": 
#     main_issue_questions()
#     main_locations_follow_up()

## Final Call Simulation

In [31]:
if __name__ == "__main__":
    main_guidelines()
    main_guidelines_follow_up()
    main_guidelines_follow_up()
    main_locations()
    main_locations_follow_up()
    main_locations_follow_up() 
    main_issue_questions()
    main_issue_questions_follow_up()

user_agent (to chat_manager):

{}

--------------------------------------------------------------------------------

Next speaker: guidelines_agent

guidelines_agent (to chat_manager):

Thank you for calling the Call Center Training 32 - Target Corporation Hotline, this is (CS Name). Would you like to file a report or follow up on an existing report?

--------------------------------------------------------------------------------
{
    "Thank you for calling the Call Center Training 32 - Target Corporation Hotline, this is (CS Name). Would you like to file a report or follow up on an existing report?": ""
}


Please provide follow-up data in JSON format (e.g. {"key": "value"} or {"key1": "value1", "key2": "value2"}):  {     "Thank you for calling the Call Center Training 32 - Target Corporation Hotline, this is (CS Name). Would you like to file a report or follow up on an existing report?": "New Report" }


user_agent (to chat_manager):

[{"content": "{}", "role": "user", "name": "user_agent"}, {"content": "Thank you for calling the Call Center Training 32 - Target Corporation Hotline, this is (CS Name). Would you like to file a report or follow up on an existing report?", "role": "user", "name": "guidelines_agent"}, {"Thank you for calling the Call Center Training 32 - Target Corporation Hotline, this is (CS Name). Would you like to file a report or follow up on an existing report?": "New Report"}]

--------------------------------------------------------------------------------

Next speaker: guidelines_agent

guidelines_agent (to chat_manager):

Before we begin, please know that this process may take 10-15 minutes or more depending on the nature of your concern or question. To ensure proper submission of your report, you will need to remain on the line until I’ve read the report narrative details back to you. I will also provide you with a report key for follow-up purposes. Do you wish

Please provide follow-up data in JSON format (e.g. {"key": "value"} or {"key1": "value1", "key2": "value2"}):  {     "Before we begin, please know that this process may take 10-15 minutes or more depending on the nature of your concern or question. To ensure proper submission of your report, you will need to remain on the line until I\u2019ve read the report narrative details back to you. I will also provide you with a report key for follow-up purposes. Do you wish to proceed?": "Yes" }


user_agent (to chat_manager):

[{"content": "{}", "role": "user", "name": "user_agent"}, {"content": "Thank you for calling the Call Center Training 32 - Target Corporation Hotline, this is (CS Name). Would you like to file a report or follow up on an existing report?", "role": "user", "name": "guidelines_agent"}, {"content": "{}", "role": "user", "name": "user_agent"}, {"content": "Thank you for calling the Call Center Training 32 - Target Corporation Hotline, this is (CS Name). Would you like to file a report or follow up on an existing report?", "role": "user", "name": "guidelines_agent"}, {"content": "[{\"content\": \"{}\", \"role\": \"user\", \"name\": \"user_agent\"}, {\"content\": \"Thank you for calling the Call Center Training 32 - Target Corporation Hotline, this is (CS Name). Would you like to file a report or follow up on an existing report?\", \"role\": \"user\", \"name\": \"guidelines_agent\"}, {\"Thank you for calling the Call Center Training 32 - Target Corporation Hotl

...

user_agent (to chat_manager):

[{"content": "{}", "role": "user", "name": "user_agent"}, {"content": "Thank you for calling the Call Center Training 32 - Target Corporation Hotline, this is (CS Name). Would you like to file a report or follow up on an existing report?", "role": "user", "name": "guidelines_agent"}, {"content": "{}", "role": "user", "name": "user_agent"}, {"content": "Thank you for calling the Call Center Training 32 - Target Corporation Hotline, this is (CS Name). Would you like to file a report or follow up on an existing report?", "role": "user", "name": "guidelines_agent"}, {"content": "[{\"content\": \"{}\", \"role\": \"user\", \"name\": \"user_agent\"}, {\"content\": \"Thank you for calling the Call Center Training 32 - Target Corporation Hotline, this is (CS Name). Would you like to file a report or follow up on an existing report?\", \"role\": \"user\", \"name\": \"guidelines_agent\"}, {\"Thank you for calling the Call Center Training 32 - Target Corporation Hotl

Please provide transcription text in JSON format (e.g. {"transcription": "I want to start filing a report!"}):  {     "Did the event occur in \"Branch Office F\" in Orlando?": "Yes" }
Please provide transcription text in JSON format (e.g. {"transcription": "I want to start filing a report!"}):  {"transcription": "I witnessed Mike using company resources for his personal side business during work hours. This misuse of resources not only violates company policy but also sets a poor example for others"}


reporter_agent (to chat_manager):

transcription = I witnessed Mike using company resources for his personal side business during work hours. This misuse of resources not only violates company policy but also sets a poor example for others
Did the event occur in "Branch Office F" in Orlando? = Yes

--------------------------------------------------------------------------------

Next speaker: classification_agent

classification_agent (to chat_manager):

{"Description": "Instances of unethical behavior or conflicts of interest, such as accepting gifts or favors in exchange for preferential treatment or using company resources for personal gain.", "Name": "Ethical Violation", "ViolationTypeId": 19}

--------------------------------------------------------------------------------

Next speaker: questions_retriever_agent

questions_retriever_agent (to chat_manager):

{
    "Where did this happen?": "",
    "What type of ethical violation occurred?": "",
    "Who was involved in the ethica

Please provide follow-up data in JSON format (e.g. {"key": "value"} or {"key1": "value1", "key2": "value2"}):  {     "Who was the witness?": "Sam" }


user_agent (to chat_manager):

[{"content": "{}", "role": "user", "name": "user_agent"}, {"content": "Thank you for calling the Call Center Training 32 - Target Corporation Hotline, this is (CS Name). Would you like to file a report or follow up on an existing report?", "role": "user", "name": "guidelines_agent"}, {"content": "{}", "role": "user", "name": "user_agent"}, {"content": "Thank you for calling the Call Center Training 32 - Target Corporation Hotline, this is (CS Name). Would you like to file a report or follow up on an existing report?", "role": "user", "name": "guidelines_agent"}, {"content": "[{\"content\": \"{}\", \"role\": \"user\", \"name\": \"user_agent\"}, {\"content\": \"Thank you for calling the Call Center Training 32 - Target Corporation Hotline, this is (CS Name). Would you like to file a report or follow up on an existing report?\", \"role\": \"user\", \"name\": \"guidelines_agent\"}, {\"Thank you for calling the Call Center Training 32 - Target Corporation Hotl

### Final Memory

In [33]:
with open("memory.json", "r") as json_file:
    chat_history_guidelines = json.load(json_file)
print(chat_history_guidelines)    

[{'content': '{}', 'role': 'user', 'name': 'user_agent'}, {'content': 'Thank you for calling the Call Center Training 32 - Target Corporation Hotline, this is (CS Name). Would you like to file a report or follow up on an existing report?', 'role': 'user', 'name': 'guidelines_agent'}, {'content': '{}', 'role': 'user', 'name': 'user_agent'}, {'content': 'Thank you for calling the Call Center Training 32 - Target Corporation Hotline, this is (CS Name). Would you like to file a report or follow up on an existing report?', 'role': 'user', 'name': 'guidelines_agent'}, {'content': '[{"content": "{}", "role": "user", "name": "user_agent"}, {"content": "Thank you for calling the Call Center Training 32 - Target Corporation Hotline, this is (CS Name). Would you like to file a report or follow up on an existing report?", "role": "user", "name": "guidelines_agent"}, {"Thank you for calling the Call Center Training 32 - Target Corporation Hotline, this is (CS Name). Would you like to file a report 